In [459]:
import numpy as np
import pandas as pd
import random

#### Load the matrices

In [460]:
battery_expenditure = pd.read_csv('battery_expenditure_matrix.csv')
distance_matrix = pd.read_csv('distance_matrix.csv')
time_matrix = pd.read_csv('time_matrix.csv')

In [461]:
customer_demands = [[3,4],[5,15],[3,7],[8,10],[6,13]]

In [462]:
covered_nodes = []

for customer_demand in customer_demands:
    covered_nodes.append(customer_demand[0])
    covered_nodes.append(customer_demand[1])
    sorted(covered_nodes)


#### Generating random combination

In [463]:
num_sequences=20
start_node=0
end_node=16
battery_threshold=10

w1=0.5
w2=0.5

In [464]:
def carCapacityMaintained(route):
    curWt = 0
    for i in range(len(route)):
        wt = route[i][1]
        curWt += wt
        if curWt > 3:
            return False
        if curWt < 0:
            return False
         
    return True


In [465]:
def allCustomersDemand(sequence):
    route=[]
    for node in sequence[1:-1]:
        route.append(node[0])
    return sorted(route) == sorted(covered_nodes)


In [466]:
def sequenceIsValid(route):
    # check for validity if every customer reaches destination
    if not allCustomersDemand(route):
        return False

    # check for validity if car strength not increase 3
    if not carCapacityMaintained(route):
        return False
    
    #check if battery level not decrease less than 0
    for i in range(len(route) - 1):
        node1 = route[i][0]
        node2 = route[i + 1][0]
                
        if node1 == 1 or node1 == 2:
            battery_level=78

            battery_consumption = battery_expenditure.iloc[node1, node2]
            battery_level -= battery_consumption
            if battery_level < battery_threshold:
                return False
    return True

In [467]:
def shuffle_array_except_first_last(arr):
    first_element = arr[0]
    last_element = arr[-1]
    internal_elements = arr[1:-1]
    random.shuffle(internal_elements)
    shuffled_array = [first_element] + internal_elements + [last_element]
    return shuffled_array

In [468]:
def generate_random_sequences(num_sequences, battery_threshold,customer_demands=customer_demands):
    sequences = []

    for _ in range(num_sequences):
        while True:
            route = [(0, 0) for _ in range(12)]
            used_positions = set()
            unused_positions = set()
            for i in range(12):
                unused_positions.add(i)
            unused_positions.remove(0)
            unused_positions.remove(11)
            used_positions.add(0)
            used_positions.add(11)

            for demand in customer_demands:
                position1 = random.randint(min(unused_positions),max(unused_positions)-1)
                while position1 in used_positions:
                    position1 = random.randint(min(unused_positions),max(unused_positions)-1)
                unused_positions.remove(position1)
                used_positions.add(position1)

                position2 = random.randint(position1 + 1, max(unused_positions))
                while position2 in used_positions:
                    position2 = random.randint(position1 + 1, max(unused_positions))
                unused_positions.remove(position2)
                used_positions.add(position2)

                route[position1] = (demand[0], +1)  # Insert the start node of the customer demand
                route[position2] = (demand[1], -1)  # Insert the end node of the customer demand

            
            if sequenceIsValid(route):
                sequences.append(route)
                break

    return sequences

In [469]:
# random_seq=generate_random_sequences(num_sequences,10)
# random_seq

In [470]:
def calculate_cost(sequence, time_matrix, distance_matrix, w1, w2):
    cost1 = 0
    cost2 = 0
    cost = 0

    for i in range(len(sequence) - 1):
        node1 = sequence[i][0]
        node2 = sequence[i + 1][0]

        cost1 += time_matrix.iloc[node1, node2]
        cost2 += distance_matrix.iloc[node1, node2]

        cost += (w1 * time_matrix.iloc[node1, node2]) + (w2 * distance_matrix.iloc[node1, node2])

    return pd.DataFrame({'Σt': [cost1], 'Σd': [cost2], 'w1*Σt+w2*Σd': [cost]})

In [471]:
cost_for_each_sequence = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2) for sequence in random_seq]
result_df = pd.concat(cost_for_each_sequence, ignore_index=True)

In [472]:
result_df

,Σt,Σd,w1*Σt+w2*Σd
0,3.430167,162.6588,83.044483
1,3.684278,173.9921,88.838189
2,3.825000,186.3056,95.065300
3,3.604917,173.1313,88.368108
4,3.530306,167.1858,85.358053
5,3.394222,161.4621,82.428161
6,3.732500,177.5077,90.620100
7,3.576083,166.2341,84.905092
8,4.028028,194.0407,99.034364
9,3.296806,153.0340,78.165403


## Genetic Algorithm implementation

In [473]:
def initialize_population(population_size):
    population = generate_random_sequences(population_size,10)
    return population


In [474]:
def tournament_selection(population, fitness_values, tournament_size):
    selected_parents = []
    for _ in range(len(population)):
        tournament_indices = np.random.choice(len(population), tournament_size, replace=False)
        tournament_fitness = [fitness_values[i] for i in tournament_indices]
        winner_index = tournament_indices[np.argmin(tournament_fitness)]
        selected_parents.append(population[winner_index])
    return selected_parents

In [475]:
import random

def ordered_crossover(parent1, parent2):
    length = len(parent1)
    crossover_point = random.randint(1, length - 2)
    

    while True:
        child = parent1[:crossover_point] + parent2[crossover_point:]
        if sequenceIsValid(child):
            break
        crossover_point = random.randint(1, length - 2)
        

    # print("Crossing : ", parent1, parent2)
    # print("Result     : ", child)

    return child


In [476]:
def shift_mutation(sequence):
    while True:
        mutation_point1 = np.random.randint(1, len(sequence) - 2)
        mutation_point2 = np.random.randint(mutation_point1 + 1, len(sequence) - 1)
        mutated_sequence = sequence[:mutation_point1] + sequence[mutation_point2:] + sequence[mutation_point1:mutation_point2]
        mutated_sequence[0] = sequence[0]
        mutated_sequence[-1] = sequence[-1]
        
        if sequenceIsValid(mutated_sequence):
            break

    print( " Original : ",sequence, " Mutated : ", mutated_sequence)
    return mutated_sequence

In [477]:
import random

def genetic_algorithm(num_generations, population_size, tournament_size, mutation_probability):
    population = initialize_population(population_size)
    
    for generation in range(num_generations):
        fitness_values = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0] for sequence in population]

        parents = tournament_selection(population, fitness_values, tournament_size)
        offspring = []

        for i in range(0, len(parents), 2):
            if i + 1 < len(parents):
                child1 = ordered_crossover(parents[i], parents[i + 1])
                child2 = ordered_crossover(parents[i + 1], parents[i])
                offspring.extend([child1, child2])

        mutated_offspring = []
        for child in offspring:
            if random.uniform(0, 1) < mutation_probability:
                mutated_child = shift_mutation(child)
                mutated_offspring.append(mutated_child)
            else:
                mutated_offspring.append(child)

        population = mutated_offspring

    best_sequence = min(population, key=lambda x: calculate_cost(x, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0])

    return best_sequence


In [478]:
best_sequence = genetic_algorithm(num_generations=100, population_size=50, tournament_size=5, mutation_probability=0.0001)
print("Best Sequence:", best_sequence)
print("Best Cost:", calculate_cost(best_sequence, time_matrix, distance_matrix, w1, w2))

KeyboardInterrupt: 